# Progetto

In [ ]:
from json import load 
from sqlite3 import connect
from pandas import DataFrame
from pandas import read_csv
from rdflib import Graph, URIRef, Literal, RDF 
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore

In [86]:
class Handler(object): #this is the first class, all the others derive from this one 

    #creating the class 
    def __init__(self, dbPathOrUrl : str):
        self.dbPathOrUrl = dbPathOrUrl

    #creating the methods 
    def getDbPathOrUrl(self): 
        return self.dbPathOrUrl 

    def setDbPathOrUrl(self, pathOrUrl : str): #: boolean 
        self.dbPathOrUrl = pathOrUrl
        return True

class UploadHandler(Handler):

    def pushDataToDb(self, path: str):  #self implied 
        if path.lower().endswith(".csv"): 
            handler = JournalUploadHandler(self.dbPathOrUrl)
            return handler.journalUpload(path) #calling the method after I called the subclass
        elif path.lower().endswith(".json"): 
            handler = CategoryUploadHandler(self.dbPathOrUrl)
            return handler.categoryUpload(path)
        else: 
            return False 


#first case: the path is of the relational database the json file

class CategoryUploadHandler(UploadHandler): 
    
    def categoryUpload(self, path: str): 

        #creating the database 
        with connect(self.dbPathOrUrl) as con: 
            con.commit() #commit the current transactions to the database  

        with open(path, "r", encoding="utf-8") as c: 
            json_data = load(c) #reading the file 

            identifier_list = []

            category_mapping_dict = {} #using it to keep track of what we have
            categories_list = []

            area_mapping_dict = {}
            area_list = []

            #internal identifier of all the items 
            for idx, item in enumerate(json_data): 
                item_internal_id = ("item-" + str(idx))
            
            #1. creating internal ids for each element: identifiers 
                identifiers = item["identifiers"] #selecting the identifiers  

                #iterating through the identifiers indise the bigger loop of items
                for idx, row in enumerate(identifiers): #i use the iteration because there are more than one in some cases 
                    identifiers_internal_id = ("internal_id-") +  str(idx)
                    identifier_list.append({
                            "item_internal_id": item_internal_id,
                            "identifier": identifiers_internal_id,
                            "identifiers": identifiers
                            })  #associating the data, with the internal id of the single category but also to the identifies of the whole item so that it's easier to query 

                #2. creating internal ids for the categories, this is trickier because they have more than one value and they can have same id
                #i have to iterate thourg everything but check if the "id" is the same, so it's useful to use a dictionary 
                categories = item["categories"]

                for idx, row in enumerate(categories): #appunto per me, scrivere cat_id = category["id"] non ha senso perchè category è una lista di un dizionario, io devo internere come dizionario il singolo item 
                    cat_id = row["id"]

                    if cat_id not in category_mapping_dict: #checking if the category is not already in the dictionary 
                        category_id_internal_id = ("category_id-") + str(idx)
                        category_mapping_dict[cat_id] = (category_id_internal_id)
                    else: 
                        category_id_internal_id = category_mapping_dict[cat_id] #if it's already inside the dict consider the original id 

                    categories_list.append({
                        "item_internal_id": item_internal_id,
                        "category_internal_id" : category_id_internal_id,
                        "id": cat_id,
                        "quartile": row["quartile"]
                    })
            
                #3. creating internal ids for areas, this is the same but without any more value 
                areas = item["areas"]

                for idx, row in enumerate(areas): 
                    area_section = areas[0]
                    if area_section not in area_mapping_dict: 
                       area_id = (("areas-") + str(idx))
                       area_mapping_dict[area_section] = area_id
                    else: 
                        area_id = area_mapping_dict[area_section]
                
                    area_list.append({
                        "item_internal_id": item_internal_id, 
                        "area_internal_id": area_id,
                        "area": area_section
                    })

            #converting the data in dataframes 
            identifiers_df = DataFrame(identifier_list)
            categories_df = DataFrame(categories_list)
            areas_df = DataFrame(area_list)

        #adding them to the database 
        with connect(self.dbPathOrUrl) as con:
            identifiers_df.to_sql("identifiers", con, if_exists="replace", index=False)
            categories_df.to_sql("categories", con, if_exists="replace", index=False)
            areas_df.to_sql("areas", con, if_exists="replace", index=False)

            
#second case: the path is the one of a graph database, the csv file


class JournalUploadHandler(UploadHandler): 
  
    def journalUpload(self, path: str):  
        my_graph = Graph() #creating the database

        #classes
        IdentifiableEntity = URIRef("https://schema.org/Thing") #I made this super generic because id is already an attribute
        Journal = URIRef("https://schema.org/Periodical") 
        Category = URIRef("https://schema.org/category")
        Area = URIRef("https://www.wikidata.org/wiki/Q26256810") #I found the one of the topic because area has a different interpretation as more of a physical meaning 

        #predicate 
        hasCategory = URIRef("http://purl.org/dc/terms/subject")
        hasArea = URIRef("https://schema.org/about")

        #attributes related to classes 
        id = URIRef("https://schema.org/identifier")
        title = URIRef("https://schema.org/title")
        languages = URIRef("https://schema.org/inLanguage") 
        publisher = URIRef("https://schema.org/publisher")
        doajSeal = URIRef("https://schema.org/Certification") 
        licence = URIRef("https://schema.org/license")
        apc = URIRef("https://schema.org/isAccessibleForFree")
        quartile = URIRef("https://schema.org/ratingValue") #to revise is it useful? 
        #the impact of the journal in the respecitive field so i use the ranking attribute


        #reading the csv  Journal title,Journal ISSN (print version),Journal EISSN (online version),Languages in which the journal accepts manuscripts,Publisher,DOAJ Seal,Journal license,APC
        journals = read_csv(path, 
                            keep_default_na=False, 
                            names=["Journal title", "Journal ISSN", "Journal EISSN", "Languages", "Publisher", "DOAJ Seal", "Journal License", "APC"],
                            dtype={
                                "Journal title": "string",
                                "Journal ISSN": "string",
                                "Journal EISSN": "string",
                                "Languages": "string",
                                "Publisher": "string",
                                "DOAJ Seal": "string",
                                "Journal License" : "string",
                                "APC": "string"
                            })
        print(journals.columns)
        #giving unique identifiers 
        base_url = "https://comp-data.github.io/res/"
        
        for idx, row in journals.iterrows(): 
            local_id = "journal-" + str(idx)
            subj = URIRef(base_url + local_id)#new local identifiers for each item in the graph database 

            my_graph.add(((subj, RDF.type, Journal))) #the subject of the row is a journal 
            #checking every category in the row (which is none other than a panda Series, so a list of vocabularies)
            if row["Journal title"]: 
                my_graph.add((subj, title, Literal(row["Journal title"])))
            if row["Journal ISSN"]: 
                my_graph.add((subj, id, Literal(row["Journal ISSN"])))
                #NEED TO DECIDE IF WE WANT TO CONSIDER BOTH AS ID, OR TO SEPATATE THEM (https://schema.org/issn) 
            if row["Journal EISSN"]: 
                my_graph.add((subj, id, Literal(row["Journal EISSN"])))
            if row["Languages"]: 
                language_string = row["Languages"] #1. taking in consideration the whole row
                language_list = language_string.split(",") #as indicated in the F.A.Q on the github they are separated with a comma but inside quotes of course ",", so I separate each item 
                for language in language_list: 
                    language = language.strip() #to delete whitespaces and facilitate the query 
                    my_graph.add((subj, languages, Literal(language)))
                
            if row["Publisher"]: 
                my_graph.add((subj, publisher, Literal(row["Publisher"])))
            if row["DOAJ Seal"]: 
                my_graph.add((subj, doajSeal, Literal(row["DOAJ Seal"])))
            if row["Journal License"]: 
                my_graph.add((subj, licence, Literal(row["Journal License"])))
            if row["APC"]: 
                my_graph.add((subj, apc, Literal(row["APC"])))

        #opening the connection to upload the graph 
        store = SPARQLUpdateStore() #initializing it as an object ù
        print(store)
        try: 
            store.open((self.dbPathOrUrl, self.dbPathOrUrl))

            for triple in my_graph.triples((None, None, None)): 
                store.add(triple)
            
            store.serialize(destination=path, format="turtle")

            #closing the connection when we finish 
            
            store.close()
        # endpoint =  self.dbPathOrUrl the endopoint is he url or path of the database 

        except Exception as e: 
            print ("Problems with the Blazegraph connection: ", e) #



        #closing the connection when we finish 
        store.close()
        #SHOULD WE WRITE SOMETHING IF IT DOESN'T WORK? print("something went wrong")
    

In [73]:
from pymantic import sparql

server = sparql.SPARQLServer('http://127.0.0.1:9999/bigdata/sparql')



In [89]:
handler = Handler(dbPathOrUrl="http://127.0.0.1:9999/")
uphandler = UploadHandler("http://127.0.0.1:9999/")

In [90]:
journalHandler = JournalUploadHandler(uphandler)
journalHandler.journalUpload("doaj.csv")

Index(['Journal title', 'Journal ISSN', 'Journal EISSN', 'Languages',
       'Publisher', 'DOAJ Seal', 'Journal License', 'APC'],
      dtype='object')
Problems with the Blazegraph connection:  unsupported operand type(s) for +: 'UploadHandler' and 'str'
